In [1]:
import sys; sys.path.append("../")

import pandas as pd

from clean.new_orleans_harbor_pd_pprr import clean_personel
from lib.columns import rearrange_personel_columns, rearrange_personel_history_columns

In [2]:
df = clean_personel()
df

,first_name,last_name,rank_desc,hire_date,term_date,hourly_salary,middle_initial,pay_effective_date,uid
0,casey,adams,police sergeant--port,1980-01-31,2010-02-28,17.00,m,1997-04-14,ac49c8119fcc397e1b00941594b68f61
1,casey,adams,police sergeant--port,1980-01-31,2010-02-28,17.68,m,2003-12-10,ac49c8119fcc397e1b00941594b68f61
2,casey,adams,police sergeant--port,1980-01-31,2010-02-28,18.39,m,2004-08-10,ac49c8119fcc397e1b00941594b68f61
3,casey,adams,police sergeant--port,1980-01-31,2010-02-28,19.13,m,2005-08-10,ac49c8119fcc397e1b00941594b68f61
4,casey,adams,police sergeant-a,1980-01-31,2010-02-28,19.13,m,2005-08-29,ac49c8119fcc397e1b00941594b68f61
...,...,...,...,...,...,...,...,...,...
910,brianne,waters,police officer 3-a,2001-03-05,2020-05-11,21.60,m,2015-10-01,d9be138289d5fdba1e41f7df1a95aefc
911,brianne,waters,police sergeant-a,2001-03-05,2020-05-11,24.34,m,2018-05-14,d9be138289d5fdba1e41f7df1a95aefc
912,brianne,waters,police sergeant-a,2001-03-05,2020-05-11,25.07,m,2018-07-15,d9be138289d5fdba1e41f7df1a95aefc
913,brianne,waters,police sergeant-a,2001-03-05,2020-05-11,25.07,m,2018-07-15,d9be138289d5fdba1e41f7df1a95aefc


In [8]:
rearrange_personel_columns(df)

,uid,last_name,middle_initial,first_name
0,ac49c8119f,adams,m,casey
1,4500bb1616,additon,p,bernard
2,3d75f7a67d,allsbrook,l,michael
3,b81578b705,anderson jr,NaN,melvin
4,25cd1eb87e,arena,p,bryan
...,...,...,...,...
113,27d88f2164,walker,NaN,winston
114,ec61f23394,walton,m,byron
115,2306549e7a,washington,m,ardiel
116,753604c68d,washington,w,ryan


In [9]:
hist = rearrange_personel_history_columns(df)
hist.rank_desc.unique()

array(['police sergeant--port', 'police sergeant-a', 'police officer 1-a',
       'police officer 2--port', 'police officer 2-a',
       'police officer 3-a', 'police officer 1--port',
       'police corporal--port', 'police lieutenant-a',
       'police lieutenant--port', 'policy planner 4',
       'police chief--port', 'police chief-a', 'admin specialist 2',
       'admin program specialist--a', 'police captain-a',
       'police captain--port'], dtype=object)